In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Add, Activation, MaxPooling2D, UpSampling2D, BatchNormalization

# Define the residual block
def residual_block(x, filters, kernel_size=3, strides=1):
    y = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size, padding='same')(y)

    # Adjust the shortcut connection to match the dimensions
    shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
    y = Add()([y, shortcut])

    y = Activation('relu')(y)
    return y

# Define the encoder with residual connection
def encoder_with_residual(input_shape):
    inputs = Input(shape=input_shape)

    # First convolution layer
    conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = Activation('relu')(conv1)

    # Residual block 1
    conv2 = residual_block(conv1, 128)

    # Residual block 2
    conv3 = residual_block(conv2, 256)

    # Residual block 3 with a residual connection
    conv4 = residual_block(conv3, 64, strides=1)

    model = tf.keras.models.Model(inputs=inputs, outputs=conv4)
    return model

# Example usage
input_shape = (3, 128, 128)  # Adjust input shape as needed
model = encoder_with_residual(input_shape)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 3, 128, 128  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_127 (Conv2D)            (None, 3, 128, 64)   73792       ['input_6[0][0]']                
                                                                                                  
 activation_39 (Activation)     (None, 3, 128, 64)   0           ['conv2d_127[0][0]']             
                                                                                                  
 conv2d_128 (Conv2D)            (None, 3, 128, 128)  73856       ['activation_39[0][0]']    

In [16]:
# Define the residual block
def residual_block(x, filters, kernel_size=3, strides=1):
    y = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size, padding='same')(y)
 
    # Adjust the shortcut connection to match the dimensions
    shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
    y = Add()([y, shortcut])  # The dimensions should now match
 
    y = Activation('relu')(y)
    return y

# def residual_block(x, filters, kernel_size=3, strides=1):
#     y = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
#     y = Activation('relu')(y)
#     y = Conv2D(filters, kernel_size, padding='same')(y)

#     # Adjust the shortcut connection to match the dimensions
#     shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
#     y = Add()([y, shortcut])

#     y = Activation('relu')(y)
#     return y


def encoder_block(x, filters):
    # Example of an encoder block
    x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), padding='same')(x)
    return x

def decoder_block(x, filters):
    # Example of a decoder block
    x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    return x

def hourglass_module(x, num_filters, depth=4):
    # Implementation of a single hourglass module
    # depth: depth of the hourglass module, controls the number of down/up sampling steps
    skips = []
    for _ in range(depth):
        x = encoder_block(x, num_filters)
        skips.append(x)
        x = residual_block(x, num_filters)
    for _ in range(depth):
        x = decoder_block(x, num_filters)
        if skips:
            print(f'Shape of X is {x.shape}; Shape of skips: {skips}')
            x = Add()([x, skips.pop()])  # Skip connections
    return x

In [28]:
def hourglass_module(x, filters):
    # First Residual Block
    x_res1 = residual_block(x, filters)

    # Downsampling
    x_downsampled = MaxPooling2D((2, 2), padding='same')(x_res1)

    # Second Residual Block
    x_res2 = residual_block(x_downsampled, filters)

    # Upsampling
    x_upsampled = UpSampling2D((2, 2))(x_res2)

    # Skip connection to the original input
    x_skip = Conv2D(filters, (1, 1), padding='same')(x)

    # Resize x_upsampled to match the shape of x_skip_resized
    x_upsampled_resized = tf.image.resize(x_upsampled, size=tf.shape(x_skip)[1:3], method='nearest')

    # Skip connection with the upsampled features
    x = Add()([x_skip, x_upsampled_resized])

    return x

def residual_block(x, filters, kernel_size=3, strides=1):
    y = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size, padding='same')(y)

    # Adjust the shortcut connection to match the dimensions
    shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
    y = Add()([y, shortcut])

    y = Activation('relu')(y)
    return y

In [49]:
def encoder_with_residual(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # First convolution layer
    conv1 = Conv2D(256, (3, 3), padding='same', activation='relu')(inputs)

    # Second convolution layer
    conv2 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv1)
#     conv2 = Activation('relu')(conv2)

    # Third convolution layer
    conv3 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv2)
#     conv3 = Activation('relu')(conv3)
    
    conv4Input = Add()([conv3, conv1])

    # Fourth convolution layer with a residual connection
    conv4 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv4Input)
#     conv4 = Activation('relu')(conv4)
    
    #Max pooling after conv4
    conv4_pooled = MaxPooling2D((3, 3))(conv4)
    print(f'Shape after pooling Layer: {conv4_pooled.shape}')
    # fifth convolution layer 
    conv5 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv4_pooled)
#     conv5 = Activation('relu')(conv5)
    
    # sixth convolution layer 
    conv6 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv5)
#     conv6 = Activation('relu')(conv6)
    
    print(f'Shape after pooling Layer: {conv6.shape}')
    
    conv7Input = Add()([conv6, conv4_pooled])
    
    # seventh convolution layer with a residual connection
    conv7 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv7Input)
#     conv7 = Activation('relu')(conv7)
    
    #Max pooling after conv7
    conv7_pooled = MaxPooling2D((3, 3), padding='same')(conv7)
    
    # eight convolution layer 
    conv8 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv7_pooled)
#     conv8 = Activation('relu')(conv8)
    
    # nineth convolution layer 
    conv9 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv8)
#     conv9 = Activation('relu')(conv9)
    
    conv10Input = Add()([conv9, conv7_pooled])
    
    #tenth convolutional layer with residual connection
    conv10 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv10Input)
#     conv10 = Activation('relu')(conv10)
    
    #max pooling after conv10
    conv10_pooled = MaxPooling2D((3, 3), padding='same')(conv10)
    
    #This needs to fed into the Hour Glass
    
    #R1
#     hourGlass1Output = hourglass_module(conv10_pooled, 64)
       # Stacked Hourglass
    hourglass = conv10_pooled
    for _ in range(4):  # You can adjust the number of stacked hourglass modules
        hourglass = hourglass_module(hourglass, 128) 
        
    for _ in range(4):  # You can adjust the number of stacked hourglass modules
        hourglass = hourglass_module(hourglass, 128) 
        
    dec_conv1 = UpSampling2D((2, 2))(hourglass)
    
    dec_conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(dec_conv1) # this output gets added to output of conv 4
    
    dec_conv3 = Conv2D(64, (3, 3), padding='same', activation='relu')(dec_conv2)
    
    dec_conv4 = Conv2D(64, (3, 3), padding='same', activation='relu')(dec_conv3)
    
    dec_conv5Input = Add()([dec_conv4, dec_conv2])
    
    dec_conv5 = Conv2D(128, (3, 3), padding='same', activation='relu')(dec_conv5Input)
    dec_conv6 = UpSampling2D((2, 2))(dec_conv5) # add this to output of conv 8
    
    dec_conv7 = Conv2D(128, (3, 3), padding='same', activation='relu')(dec_conv6)
    dec_conv8 = Conv2D(128, (3, 3), padding='same', activation='relu')(dec_conv7)
    
    dec_conv9Input = Add()([dec_conv8, dec_conv6])
    
    dec_conv9 = Conv2D(256, (3, 3), padding='same', activation='relu')(dec_conv9Input)
    dec_conv10 = UpSampling2D((2, 2))(dec_conv9) # add this to output of conv 12
    
    dec_conv11 = Conv2D(256, (3, 3), padding='same', activation='relu')(dec_conv10)
    dec_conv12 = Conv2D(256, (3, 3), padding='same', activation='relu')(dec_conv11)
    
    dec_conv13Input = Add()([dec_conv12, dec_conv10])
    dec_conv13 = Conv2D(256, (3, 3), padding='same', activation='relu')(dec_conv13Input)
    
    
    model = tf.keras.models.Model(inputs=inputs, outputs=conv4)
    return model

# Example usage
input_shape = (3, 128, 128)  # Adjust input shape as needed
model = encoder_with_residual(input_shape)
model.summary()

Shape after pooling Layer: (None, 1, 42, 128)
Shape after pooling Layer: (None, 1, 42, 128)
Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_34 (InputLayer)          [(None, 3, 128, 128  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1631 (Conv2D)           (None, 3, 128, 256)  295168      ['input_34[0][0]']               
                                                                                                  
 conv2d_1632 (Conv2D)           (None, 3, 128, 256)  590080      ['conv2d_1631[0][0]']            
                                                                                                  

# Training Model

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras.optimizers import Adam

# # Assuming you have a labeled dataset: train_images and train_labels

# # Define the model
# input_shape = (128, 128, 3)
# model = encoder_decoder(input_shape)

# # Compile the model
# model.compile(optimizer=Adam(), loss=MeanSquaredError())

# # Train the model
# model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# # Save the trained model
# model.save("encoder_decoder_model.h5")